# 多帧图像转为视频

In [ ]:
import cv2
import os

# 图像文件夹路径
image_folder = "visualResult/CoDeFv1/beauty_1"
fps = 30.0

# 视频输出路径和名称
video_name = "visualResult/CoDeFv1/beauty_1.avi"

# 获取图像文件夹中的所有图像文件名
images = [img for img in os.listdir(image_folder)]

# 读取第一张图像，获取图像尺寸
frame = cv2.imread(os.path.join(image_folder, images[0]))
height, width, layers = frame.shape

# 使用cv2.VideoWriter创建视频写入对象
video = cv2.VideoWriter(video_name, cv2.VideoWriter_fourcc(*"MJPG"), fps, (width, height))

# 将每个图像逐帧写入视频
for image in images:
    video.write(cv2.imread(os.path.join(image_folder, image)))

# 关闭视频写入对象
video.release()


# 采样图像尺寸为指定大小

In [ ]:
# 应用PIL
from PIL import Image

def resize_image(input_image_path, output_image_path, size):
    with Image.open(input_image_path) as image:
        resized_image = image.resize(size)
        resized_image.save(output_image_path)

# 示例用法
input_path = 'input.jpg'  # 输入图像路径
output_path = 'output.jpg'  # 输出图像路径
target_size = (800, 600)  # 目标大小，宽度为800像素，高度为600像素

resize_image(input_path, output_path, target_size)


In [ ]:
# 应用opencv
# 指定新的图像尺寸
import cv2

# 加载图像
image = cv2.imread('input_image.jpg')  # 替换为你的图像路径

new_width = 500
new_height = 300

# 采样图像
resized_image = cv2.resize(image, (new_width, new_height))
# 保存采样后的图像
cv2.imwrite('output_image.jpg', resized_image)  # 替换为你想保存的路径和文件名

